In [ ]:
# import json
# import pandas as pd
# with open('1_input_data/config.json', 'r') as file:
#         config = json.load(file)
# image_opportunity_df = pd.read_csv(config["csv_file_path"]["image_capture_opportunity"])

# image_opportunity_df['SatID'] = image_opportunity_df['SatID'].astype(str)
# image_opportunity_df['OpportunityStartOffset'] = image_opportunity_df['OpportunityStartOffset'].astype(int)
# image_opportunity_df['OpportunityEndOffset'] = image_opportunity_df['OpportunityEndOffset'].astype(int)
# image_opportunity_df_copy = image_opportunity_df.copy()
# image_opportunity_df_copy['X'] = image_opportunity_df_copy[['OpportunityStartTime','OpportunityStartOffset']].apply(lambda a: pd.to_datetime(a['OpportunityStartTime']) - pd.DateOffset(seconds=a['OpportunityStartOffset']),axis=1)
# image_opportunity_df_copy['Y'] = image_opportunity_df_copy[['OpportunityEndTime','OpportunityEndOffset']].apply(lambda a: pd.to_datetime(a['OpportunityEndTime']) - pd.DateOffset(seconds=a['OpportunityEndOffset']),axis=1)
# base_time_stamp = image_opportunity_df_copy["X"].to_list()[0]

# image_opportunity_df['base_time'] = base_time_stamp
# image_opportunity_df['req_date'] = image_opportunity_df[['base_time','OpportunityStartOffset']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['OpportunityStartOffset']),axis=1)#pd.to_datetime(image_opportunity_df['base_time']) + pd.to_timedelta(image_opportunity_df['OpportunityStartOffset'])
# image_opportunity_df[['req_date','OpportunityStartTime','base_time','OpportunityStartOffset']]

# image_opportunity_df['x'] = image_opportunity_df[['Priority','StripID']].apply(lambda a: a['Priority'] if a['Priority']<=0 else a['StripID'],axis=1)
# image_opportunity_df[['Priority','StripID','x']]


In [1]:
#!pip list
import os 
os.getcwd()

import sys
print(sys.path)

sys.path.append("../")
print(sys.path)


['/Users/robin/Documents/Schedule_plan/git_APS_Python_core_develop/APS_Python_core/src/APS_Python_core', '/opt/anaconda3/lib/python312.zip', '/opt/anaconda3/lib/python3.12', '/opt/anaconda3/lib/python3.12/lib-dynload', '', '/opt/anaconda3/lib/python3.12/site-packages', '/opt/anaconda3/lib/python3.12/site-packages/aeosa']
['/Users/robin/Documents/Schedule_plan/git_APS_Python_core_develop/APS_Python_core/src/APS_Python_core', '/opt/anaconda3/lib/python312.zip', '/opt/anaconda3/lib/python3.12', '/opt/anaconda3/lib/python3.12/lib-dynload', '', '/opt/anaconda3/lib/python3.12/site-packages', '/opt/anaconda3/lib/python3.12/site-packages/aeosa', '../']


In [2]:
#image_opportunity_df.columns
result_dict = {'a':2,'b':66,'c':43}
result_dict.update({'d':55,'ww':222})
result_dict

{'a': 2, 'b': 66, 'c': 43, 'd': 55, 'ww': 222}

In [3]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import json
import math

from datetime import datetime as dt
import os
# from APS_Python_core.preprocess_1.preprocess_GSpassSelecion import GSPassPreprocess
# from model_3.MILP_GSpassSelection_v3 import GSpassSelection
# from postprocess_4.postprocess_GSpassSelection import GSpasspostprocess

# from preprocess_1.preprocess_imageAqusuition_test import ImageAquisitionProcess #preprocess_imageAqusuition_test,preprocess_imageAquisition_v3_18112024
# from model_3.MILP_imageCapture_v3_17112024 import ImageCapturePlan # MILP_imageCapture_v2_16102024,MILP_imageCapture_v2_25102024 # MILP_imageCapture_v2_07112024 #MILP_imageCapture_v3_17112024
# from postprocess_4.image_capture_postprocess_V3_17112024 import ImagecapturePostProcess # image_capture_postprocess_V3_17112024# image_capture_postprocess_v2_18102024

# from preprocess_1.preprocess_downlink_WIP import DownlinkingPreProcess
# from model_3.MILP_downlink import ImageDownlinkPlan
# from postprocess_4.postprocess_downlink import ImageDownlinkPostProcess

# from result_interpret import interpret_result
# from utils import *

from APS_Python_core.preprocess_1.preprocess_GSpassSelecion import GSPassPreprocess
from APS_Python_core.model_3.MILP_GSpassSelection_v3 import GSpassSelection
from APS_Python_core.postprocess_4.postprocess_GSpassSelection import GSpasspostprocess

from APS_Python_core.preprocess_1.preprocess_imageAqusuition_test import ImageAquisitionProcess #preprocess_imageAqusuition_test,preprocess_imageAquisition_v3_18112024
from APS_Python_core.model_3.MILP_imageCapture_v3_17112024 import ImageCapturePlan # MILP_imageCapture_v2_16102024,MILP_imageCapture_v2_25102024 # MILP_imageCapture_v2_07112024 #MILP_imageCapture_v3_17112024
from APS_Python_core.postprocess_4.image_capture_postprocess_V3_17112024 import ImagecapturePostProcess # image_capture_postprocess_V3_17112024# image_capture_postprocess_v2_18102024

from APS_Python_core.preprocess_1.preprocess_downlink_WIP import DownlinkingPreProcess
from APS_Python_core.model_3.MILP_downlink import ImageDownlinkPlan
from APS_Python_core.postprocess_4.postprocess_downlink import ImageDownlinkPostProcess

from APS_Python_core.result_interpret import interpret_result
from APS_Python_core.utils import *

# script_dir = os.path.abspath( os.path.dirname( __file__ ) )
# print("script directory: ",script_dir)
'''
readout is happening after the last image and before entering the eclipse region.
Any heating operation will start if the temp reaches around intial tempertaure.
Higher number of Global priority is assumed to be Higher prior Important image.
if end date of due date is less than 24 hrs from the reference time offset then it is going to assured tasking.
Need offset from when scheduling is started. Example if scheduling is  needed from 10:00AM to 11 PM . Then offset is needed from 10:00 AM or 09:59 AM. Due date based assured tasking is the reason.
Iniial Camera Memory is needed at the start of the any oppr imaging/gsPass whichever is first. 
Iniial Readout Memory is needed at the start of the readout oppr.
Initial power is needed at start of the opportunity(imaging/gspass).For Now (since power constraint is not there for readout)
Iniial thermal value is needed before the start of the oppr imaging/gsPass/readout according to device (for NCCms:readout , for camera detector: Imaging ,For XBT : gs Pass oppr).
Eclipse Event should be starting from first oppr either gsPass/Imaging. Readout is happening after the first imaging so i guess not needed at readout.
'''
'''
Need offset from when scheduling is started. Example if scheduling is  needed from 10:00AM to 11 PM . Then offset is needed from 10:00 AM or 09:59 AM.
Iniial Memory,power and thermal value is needed at 10:00 AM
'''
def select_gs_pass_oppr(GS_pass_df,config):

    obj_preprocess = GSPassPreprocess(GS_pass_df)
    data = obj_preprocess.preprocess()

    obj_model = GSpassSelection(data,config)
    result,thermal_profile_gsPass = GSpasspostprocess(obj_model,data,config).get_gsPasses()# 21 seconds

    try :
        result['duration'] = result['end_time'] - result['start_time']
        result = result[result['duration']> 0]
    except:
        print("model is not converged or infeasible or not solved")

    return result
    

def select_img_opprtunity(image_opportunity_df,gs_pass_result_df,eclipse_df_dict,config):

    #basic flters
    #image_opportunity_df = image_opportunity_df[image_opportunity_df['OpportunityEndOffset']<config['scheduled_Hrs']*3600]
    image_opportunity_df = image_opportunity_df[image_opportunity_df['CloudCoverLimit']>image_opportunity_df['CloudCover']]
    image_opportunity_df = image_opportunity_df[image_opportunity_df['OffNadirLimit']>image_opportunity_df['OffNadir']]

    obj_preprocess = ImageAquisitionProcess(image_opportunity_df,gs_pass_result_df,eclipse_df_dict,config)
    data = obj_preprocess.preprocess()
    #print(data['cs1j2k2Domainlist__cs1j1k1'])

    #++++++++++++++++++++++++++  STEP 0  +++++++++++++++++++++++++++++++++++++++++++++++
    '''
    hard code some data
    '''
    data['camera_memory_capacity__s'] = {s:v for s,v in data['camera_memory_capacity__s'].items() }
    data['readout_memory_capacity__s'] = {s:v for s,v in data['readout_memory_capacity__s'].items() }
    data['power_capacity__s']  = {s:720000000 for s,v in data['power_capacity__s'].items() }
    data['initial_power_value__s']  = {s:v*0.3 for s,v in data['power_capacity__s'].items() }
    #++++++++++++++++++++++++++  STEP 1  +++++++++++++++++++++++++++++++++++++++++++++++
    config['objective']['GS_Pass_time'] = True
    config['objective']['total_priority'] = False
    config['objective']['total_readout_memory'] = False
    obj_model = ImageCapturePlan(data,config)

    #Readout Schedule 
    # data['GS_Pass_time_objective'] = obj_model.prob.objective.value()
    # config['objective']['GS_Pass_time'] = False
    # config['objective']['total_priority'] = False
    # config['objective']['total_readout_memory'] = True
    # obj_model = ImageCapturePlan(data,config)

    #++++++++++++++++++++++++++  STEP 2  +++++++++++++++++++++++++++++++++++++++++++++++

    # config['objective']['GS_Pass_time'] = False
    # config['objective']['total_priority'] = True
    # #config['objective']['total_readout_memory'] = False

    # obj_model = ImageCapturePlan(data,config)

    # data['total_priority_objective'] = obj_model.prob.objective.value()
    #++++++++++++++++++++++++++  STEP 3  +++++++++++++++++++++++++++++++++++++++++++++++

    # config['objective']['total_priority'] = False
    # config['objective']['total_readout_memory'] = True

    # obj_model = ImageCapturePlan(data,config)
    #++++++++++++++++++++++++++  PostProcess  +++++++++++++++++++++++++++++++++++++++++++++++
    post_obj = ImagecapturePostProcess(obj_model,data)
    img_capture_result= post_obj.get_schedule()
    #.isnull().sum()
    return img_capture_result,data
    

    #======================================================================================================================================================================================================

    pass

def get_aps_success_metric(img_capture_result,data):
    after_aps_plan_df = img_capture_result[img_capture_result['operation']=='Imaging']
    criteria_list = ['total_opprtunities_ratio','GP_ratio','LP_ratio','conflictImg_gsPass']

    total_conflict_images_list= [data['success_metric_before']['conflict_images']]
    GP_before = data['success_metric_before']['original_Total_GP']
    LP_before = data['success_metric_before']['original_Total_LP']
    TOppr_before = data['success_metric_before']['total_opportunities'] 
    before_list = [TOppr_before,GP_before,LP_before,total_conflict_images_list]

    GP_after= after_aps_plan_df['mean_global_priority'].sum()
    LP_after = after_aps_plan_df['mean_local_priority'].sum()
    TOppr_after= after_aps_plan_df['encoded_strip_id'].nunique()
    fraction_conflict_images_list = [list(after_aps_plan_df[after_aps_plan_df['encoded_strip_id'].isin(total_conflict_images_list)]['encoded_strip_id'].unique())]
    after_list = [TOppr_after,GP_after,LP_after,fraction_conflict_images_list]

    APS_success_metric_df = pd.DataFrame({'criteria':criteria_list,'potential_input':before_list,'APS_selected':after_list})
    #APS_success_metric_df['percentage'] = APS_success_metric_df['APS_selected'] / APS_success_metric_df['potential_input']
    APS_success_metric_df1 = APS_success_metric_df[:-1]
    APS_success_metric_df1['percentage'] = APS_success_metric_df1['APS_selected'] / APS_success_metric_df1['potential_input'] * 100
    APS_success_metric_df2 = APS_success_metric_df[-1:]

    APS_success_metric_df = pd.concat([APS_success_metric_df1,APS_success_metric_df2])

    return APS_success_metric_df
    

def get_downlink_schedule(image_downlink_df,img_capture_result,config):

    downlink_operation_list  = ['downlinking_from_camera','downlinking_from_Readout']
    img_capture_result_downlink = img_capture_result[img_capture_result['operation'].isin(downlink_operation_list)]
    DownlinkingPreProcessObj = DownlinkingPreProcess(image_downlink_df,img_capture_result_downlink,config)
    data_downlink = DownlinkingPreProcessObj.preprocess()
    
    if config['downlink_schedule_OnlyJustsortImgID']:
        downlink_result = pd.DataFrame(data_downlink['LP_DD_Priority_imgID'].items(),columns=['ImageID','computed_priority']).\
            sort_values(by='computed_priority',ascending=False)
    else:
        obj_downlink_model = ImageDownlinkPlan(data_downlink,config)
        downlink_result = ImageDownlinkPostProcess(obj_downlink_model,data_downlink).get_schedule()
        downlink_result = downlink_result[downlink_result['TileStripNo_downLoad']!=0]

    return downlink_result
    
def schedule():

    pass

def get_input_files(config,GS_pass_df,image_opportunity_df,image_downlink_df):
    # GS PASS
    #GS_pass_df = pd.read_csv(config["csv_file_path"]["gs_pass_opportunity"])#APS_gsPasses_TV1#GS_Passes_mock1#GS_Passes_live1#GS_Passes_new (1)
    
    #GS_pass_df = pd.read_csv("1_input_data/GS_Passes_new (1).csv")
    GS_pass_df_original = GS_pass_df.copy()
    GS_pass_df['SatID'] = GS_pass_df['SatID'].astype(str)
    GS_pass_df['AOSOffset'] = GS_pass_df['AOSOffset'].astype(int)
    GS_pass_df['LOSOffset'] = GS_pass_df['LOSOffset'].astype(int)

    # image Opprtunity
    #image_opportunity_df = pd.read_csv(config["csv_file_path"]["image_capture_opportunity"])#Imaging_mock1#APS_imagingOpportunities_TV1#Imaging_live#Imaging_new (1)
    # change made priority ulta
    #image_opportunity_df['Priority'] = 1/image_opportunity_df['Priority']
    
    image_opportunity_df['SatID'] = image_opportunity_df['SatID'].astype(str)
    image_opportunity_df['OpportunityStartOffset'] = image_opportunity_df['OpportunityStartOffset'].astype(int)
    image_opportunity_df['OpportunityEndOffset'] = image_opportunity_df['OpportunityEndOffset'].astype(int)
    image_opportunity_df_copy = image_opportunity_df.copy()
    image_opportunity_df_copy['X'] = image_opportunity_df_copy[['OpportunityStartTime','OpportunityStartOffset']].apply(lambda a: pd.to_datetime(a['OpportunityStartTime']) - pd.DateOffset(seconds=a['OpportunityStartOffset']),axis=1)
    image_opportunity_df_copy['Y'] = image_opportunity_df_copy[['OpportunityEndTime','OpportunityEndOffset']].apply(lambda a: pd.to_datetime(a['OpportunityEndTime']) - pd.DateOffset(seconds=a['OpportunityEndOffset']),axis=1)
    base_time_stamp = image_opportunity_df_copy["X"].to_list()[0]
    config['base_time_stamp_downlink'] = base_time_stamp

    #image Downlink
    #image_downlink_df = pd.read_csv(config["csv_file_path"]["image_downlink_file"])
    image_downlink_df['assured_downlink_flag'] = [0,0] +[0]*(len(image_downlink_df)-2)
    image_downlink_df['delivery_type'] = 'standard_delivery' # expedited_delivery,super_expedited_delivery
    union_list_of_sat = list(set(image_opportunity_df['SatID']).union(set(GS_pass_df['SatID'])).union(set(image_downlink_df['SatID'])))
    hrs = config['scheduled_Hrs']

    # get dummy eclipse data close to reality
    #eclipse_df = pd.DataFrame()
    #for sat in satellite_list:
        #this_eclipse_df = eclipse_event_df[eclipse_event_df['SatID']==sat]
        #that_eclipse_df = get_eclipse_data(this_eclipse_df,config)
        #eclipse_df = pd.concat([that_eclipse_df,eclipse_df])
        
    min_time_index= min([image_opportunity_df['OpportunityStartOffset'].min(),image_opportunity_df['OpportunityEndOffset'].max(),GS_pass_df['AOSOffset'].min(),GS_pass_df['LOSOffset'].max()])
    max_time_index= max([image_opportunity_df['OpportunityStartOffset'].min(),image_opportunity_df['OpportunityEndOffset'].max(),GS_pass_df['AOSOffset'].min(),GS_pass_df['LOSOffset'].max()])

    hrs = (max_time_index - min_time_index)/3600
    hrs = math.ceil(hrs)
    while True:
        hrs += 1
        if hrs % 1.5==0:
            break


    in_orbit_eclipse_event = [1 for i in range(int(1.5*3600*0.6))] + [0 for i in range(int(1.5*3600*0.4))] #
    eclipse_df  = pd.DataFrame({'time_index': [i for i in range(min_time_index,min_time_index+hrs*3600)] ,"eclipse" : in_orbit_eclipse_event*int(hrs/1.5)})
    eclipse_df['SatID']= [union_list_of_sat] *len(eclipse_df)
    eclipse_df = eclipse_df.explode('SatID')
    eclipse_df_dict = {s: eclipse_df[eclipse_df['SatID']==s] for s in eclipse_df['SatID'].unique()}

    #eclipse_df = pd.DataFrame()
    #for sat in satellite_list:
        #this_eclipse_df = eclipse_event_df[eclipse_event_df['SatID']==sat]
        #that_eclipse_df = get_eclipse_data(this_eclipse_df,config)
        #eclipse_df = pd.concat([that_eclipse_df,eclipse_df])
   #eclipse_df_dict = {s: eclipse_df[eclipse_df['SatID']==s] for s in eclipse_df['SatID'].unique()}

    
    # get dummy data for assured tasking
    image_opportunity_df['encoded_stripId'] =   image_opportunity_df['StripID'].astype(str)+ '_' + image_opportunity_df['AoiID'].astype(str)
    total_capture_list = list(image_opportunity_df['encoded_stripId'].unique())
    no_of_list = len(total_capture_list)
    assured_capture_df = pd.DataFrame({'encoded_stripId':total_capture_list,'assured_task':[0,0]+[0]*(no_of_list-2)})
    image_opportunity_df = pd.merge(image_opportunity_df,assured_capture_df,on='encoded_stripId',how='left')
    image_opportunity_df = image_opportunity_df.drop(columns=['encoded_stripId'])

    # further processing eclipse data to align with gs pass where entire gs pass is assumed to be in eclipse region
    gsPassInput_df_copy = GS_pass_df_original
    gsPassInput_df_copy['SatID'] = gsPassInput_df_copy['SatID'].astype(str)
    gsPassInput_df_copy['AOSOffset'] = gsPassInput_df_copy['AOSOffset'].astype(int)
    gsPassInput_df_copy['LOSOffset'] = gsPassInput_df_copy['LOSOffset'].astype(int)
    gsPassInput_df_copy['list'] =  gsPassInput_df_copy[['AOSOffset','LOSOffset']].apply(lambda a : [i for i in range(a['AOSOffset'],a['LOSOffset']+1)],axis =1 )

    gsPassInput_df_copy1 = gsPassInput_df_copy[['SatID','list']]
    gsPassInput_df_copy1 = gsPassInput_df_copy1.explode('list')
    gsPassInput_df_grouped_copy1 = gsPassInput_df_copy1.groupby('SatID').agg(time_index_list = ('list',list)).reset_index()
    gsPasstimeIndexList__s = dict(zip(gsPassInput_df_grouped_copy1['SatID'],gsPassInput_df_grouped_copy1['time_index_list']))
    for k,v in eclipse_df_dict.items():
        if k in gsPasstimeIndexList__s.keys():
            this_time_index_list = gsPasstimeIndexList__s[k]
            v.loc[v["time_index"].isin(this_time_index_list), "eclipse"] = 1
            eclipse_df_dict[k] = v

    return {
            'GS_pass_df':GS_pass_df,\
            'image_opportunity_df':image_opportunity_df,\
            'image_downlink_df':image_downlink_df,\
            "eclipse_df_dict": eclipse_df_dict,
            "config":config
            }
    
def get_schedule(config,GS_pass_df,image_opportunity_df,image_downlink_df):

    # Open and read the JSON file
    #APS_Python_core/src/APS_Python_core/1_input_data/config.json
    #with open('APS_Python_core/src/APS_Python_core/1_input_data/config.json', 'r') as file:
    #with open('../1_input_data/config.json', 'r') as file:
        #config = json.load(file)
    original_image_opportunity_df = image_opportunity_df.copy()
    # if memory constraint False then thermal_constraint is also False
    config['constraints']['thermal_constraint_readout'] = config['constraints']['memory_constrant'] and config['constraints']['thermal_constraint_readout']
    config['constraints']['thermal_constraint_imaging'] = config['constraints']['memory_constrant'] and config['constraints']['thermal_constraint_imaging']

    #======================================================================================================================================================================================================
    # read_input
    input_dict = get_input_files(config,GS_pass_df,image_opportunity_df,image_downlink_df)
    config = input_dict['config']
    #======================================================================================================================================================================================================
    #gs pass_selection
    gs_pass_result_df = select_gs_pass_oppr(input_dict['GS_pass_df'],config)
    gs_pass_result_df['Eclipse'] = 1 ## dummy
    gs_pass_result_df['duration'] = gs_pass_result_df['end_time'] - gs_pass_result_df['start_time']
    gs_pass_result_df = gs_pass_result_df[gs_pass_result_df['duration']> 0]
    interpret_gs_pass_result_df_copy = gs_pass_result_df.copy()# this not the gsPass result as it is to be get filtered after due to other factors in image capture plan.It is just to get require info in interpret result.

    print("image_capture_plan_starting")
    #======================================================================================================================================================================================================
    #image_selection
    img_capture_result,capture_plan_data_input= select_img_opprtunity(input_dict['image_opportunity_df'],gs_pass_result_df,input_dict['eclipse_df_dict'],config)
    img_capture_result['base_time'] = config['base_time_stamp_downlink']
    #img_capture_result = img_capture_result[img_capture_result['operation']=='Imaging']
    #readout_result = img_capture_result[img_capture_result['operation']=='Readout']
    interpret_img_capture_resul_copy = img_capture_result.copy()
    #======================================================================================================================================================================================================
    # get APS success metrics 
    APS_success_metric_df = get_aps_success_metric(img_capture_result,capture_plan_data_input)
    #======================================================================================================================================================================================================
    print("Downlink_plan_starting")
    downlink_result = get_downlink_schedule(input_dict['image_downlink_df'],img_capture_result,config)
    downlink_result['base_time'] = config['base_time_stamp_downlink']
    #======================================================================================================================================================================================================
    #img_capture_result[img_capture_result['download_from_']]
    # gs_pass_result_df.to_csv("APS_Python_core/src/APS_Python_core/5_output_data/gs_pass_result_df.csv",index=None)
    # img_capture_result.to_csv("APS_Python_core/src/APS_Python_core/5_output_data/img_capture_schedule.csv",index=None)
    # APS_success_metric_df.to_csv("APS_Python_core/src/APS_Python_core/5_output_data/APS_success_metric.csv",index = None)
    # downlink_result.to_csv("APS_Python_core/src/APS_Python_core/5_output_data/downlink_result.csv",index = None)

    interpret_image_opportunity_df = original_image_opportunity_df
    interpret_result_dict = interpret_result(interpret_image_opportunity_df,interpret_gs_pass_result_df_copy,interpret_img_capture_resul_copy,config)
    for k,v in interpret_result_dict.items():
        v['base_time'] = config['base_time_stamp_downlink']
        #v.to_csv("APS_Python_core/src/APS_Python_core/5_output_data/"+k+".csv",index = None)
    
    only_img_capture_result = img_capture_result[img_capture_result['operation']=='Imaging'][['SatID','start_time','end_time','AoiID','StripID','base_time']]
    only_img_capture_result['start_time'] = only_img_capture_result[['start_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['start_time']),axis=1)
    only_img_capture_result['end_time'] = only_img_capture_result[['end_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['end_time']),axis=1)

    only_readout_result = img_capture_result[img_capture_result['operation']=='Readout'][['SatID','start_time','end_time','base_time']]
    only_readout_result['start_time'] = only_readout_result[['start_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['start_time']),axis=1)
    only_readout_result['end_time'] = only_readout_result[['end_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['end_time']),axis=1)

    only_gsPass_result = img_capture_result[img_capture_result['operation']=='downlinking_from_Readout'][['SatID','start_time','end_time','gsID','base_time']]
    only_gsPass_result['start_time'] = only_gsPass_result[['start_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['start_time']),axis=1)
    only_gsPass_result['end_time'] = only_gsPass_result[['end_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['end_time']),axis=1)


    result_dict = {"only_readout_result":only_readout_result,\
                  "only_img_capture_result":only_img_capture_result,\
                  "only_gsPass_result":only_gsPass_result}
    result_dict.update(interpret_result_dict)
    #return result_dict
    return result_dict

    #print(only_img_capture_result,only_readout_result,downlink_result)
#config['constraints'] = ['Thermal_constraints']

In [4]:
GS_pass_df = GS_pass_df = pd.read_csv("1_input_data/GS_Passes_new (1).csv")
image_opportunity_df = pd.read_csv("1_input_data/Imaging_new (1) copy.csv")
image_downlink_df = pd.read_csv("1_input_data/APS_imageTable_TV1.csv")
with open('1_input_data/config.json', 'r') as file:
    config = json.load(file)
result_dict = get_schedule(config,GS_pass_df,image_opportunity_df,image_downlink_df)


start_solving
Running HiGHS 1.8.1 (git hash: 4a7f24a): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+09]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+00]
  RHS    [9e+04, 1e+09]
Presolving model
335 rows, 250 cols, 750 nonzeros  0s
169 rows, 167 cols, 418 nonzeros  0s
77 rows, 75 cols, 188 nonzeros  0s

Solving MIP model with:
   77 rows
   75 cols (37 binary, 0 integer, 0 implied int., 38 continuous)
   188 nonzeros
MIP-Timing:     0.00065 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol          

In [ ]:
#max(image_opportunity_df['Priority'].to_list())

In [ ]:
result_dict.keys()

In [5]:
result_dict['only_img_capture_result']['StripID'].nunique(),len(result_dict['only_gsPass_result'])

(26, 83)

In [7]:
#capture_plan_data_input['']
#-48283.5824309
#-48271
len(result_dict['interpret_selected_oppr_conflict_comparision_df'])

26

In [ ]:
result_dict['interpret_selected_oppr_conflict_comparision_df']#['conflic_strip_flag_named'].nunique()##['interpret_selected_oppr_conflict_comparision_df']#['interpret_extracted_raw_file_df']

In [ ]:
#downlink_result

In [ ]:
l1 = 'extracted_raw_file_df', 'selected_oppr_conflict_comparision_df', 'KPI_df'
df111 = result_dict['extracted_raw_file_df']#.columns
#interpret_result_dict.keys()


In [ ]:
Y = img_capture_result[img_capture_result['operation']=='Imaging']#['StripID']#.nunique()
Z = img_capture_result[img_capture_result['operation']=='downlinking_from_Readout']
Z['duration'] = Z['end_time']- Z['start_time']
#Y[Y['StripID']=='']

#Y[Y['encoded_strip_id']=='Order 1 - Strip 0_Area 0']
Y['StripID'].nunique(),len(Y),len(Z),Z['duration'].sum(),gs_pass_result_df['duration'].sum(),len(gs_pass_result_df)


In [ ]:
Y['SatID'].unique()

In [ ]:
CI = Y[Y['SatID']=='FF03']
RZO = img_capture_result[img_capture_result['operation']=='Readout']
CR = RZO[RZO['SatID']=='FF03']
pd.concat([CI,CR]).sort_values(by='start_time')



In [ ]:
CR

In [ ]:
#'Order 1 - Strip 0_Area 0'
RZO = img_capture_result[img_capture_result['operation']=='Readout']
RZO[RZO['SatID']=='FF01']
RZO



In [ ]:
RZ = capture_plan_data_input['dedicated_readout_df']
len(RZ[RZ['SatID']=='FF01'].sort_values(by='start_time')),len(RZ[RZ['SatID']=='FF02'].sort_values(by='start_time')),len(RZ[RZ['SatID']=='FF03'].sort_values(by='start_time'))
RZ[RZ['SatID']=='FF01'].sort_values(by='start_time')

In [ ]:
img_capture_result['operation'].unique()

[ item for item in capture_plan_data_input['Memory_NoimageGs_TW_list'] if item[2]=='FF01']
#capture_plan_data_input['dedicatedReadoutTWlist__concat_sat_memoryTWindex']['FF01_136.0']

In [ ]:
capture_plan_data_input['active_assured_strip_id_list']

In [ ]:
#filtered_eclipse_data.csv




In [13]:
#eclipse_df[eclipse_df['eclipse']==0]

In [ ]:
eclipse_df.sort_values(by='time_index')

In [ ]:
#config

In [ ]:
#SatID,start_time,end_time,eclipse

In [ ]:
#['GS_pass_df', 'image_opportunity_df', 'image_downlink_df', 'eclipse_df_dict', 'config']
#input_dict['eclipse_df_dict']['FF02']

In [ ]:
capture_plan_data_input['readout_memory_capacity__s']

In [ ]:
capture_plan_data_input['imgery_sat_id_list']#.keys()
[s+'_'+str(n) for s in capture_plan_data_input['imgery_sat_id_list']+capture_plan_data_input['only_gs_sat_id_list'] \
 if s in capture_plan_data_input['dedicatedReadoutTWIndex__sat'].keys() for n in capture_plan_data_input['dedicatedReadoutTWIndex__sat'][s]]



In [ ]:
#img_capture_result[img_capture_result['operation']=='Imaging']

#eclipse_df_dict

In [ ]:
capture_plan_data_input['active_assured_strip_id_list']

In [ ]:
capture_plan_data_input['assured_tasking_based_on_input_list']

In [ ]:
capture_plan_data_input['cs1j2k2Domainlist__cs1j1k1']['FF01_Order 1 - Strip 0_Area 0_1.0']

In [9]:
capture_plan_data_input['GS_Pass_time_objective'] = 0

In [ ]:
#len(img_capture_result[img_capture_result['operation']=='Imaging'])

In [ ]:
capture_plan_data_input

## Run till here

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import json
import math

from datetime import datetime as dt

import os
from preprocess_1.preprocess_GSpassSelecion import GSPassPreprocess
from model_3.MILP_GSpassSelection_v3 import GSpassSelection
from postprocess_4.postprocess_GSpassSelection import GSpasspostprocess

from preprocess_1.preprocess_imageAqusuition_test import ImageAquisitionProcess #preprocess_imageAqusuition_test,preprocess_imageAquisition_v3_18112024
from model_3.MILP_imageCapture_v3_17112024 import ImageCapturePlan # MILP_imageCapture_v2_16102024,MILP_imageCapture_v2_25102024 # MILP_imageCapture_v2_07112024 #MILP_imageCapture_v3_17112024
from postprocess_4.image_capture_postprocess_V3_17112024 import ImagecapturePostProcess # image_capture_postprocess_V3_17112024# image_capture_postprocess_v2_18102024

from preprocess_1.preprocess_downlink_WIP import DownlinkingPreProcess
from model_3.MILP_downlink import ImageDownlinkPlan
from postprocess_4.postprocess_downlink import ImageDownlinkPostProcess

from result_interpret import interpret_result
from utils import *

# script_dir = os.path.abspath( os.path.dirname( __file__ ) )
# print("script directory: ",script_dir)
'''
readout is happening after the last image and before entering the eclipse region.
Any heating operation will start if the temp reaches around intial tempertaure.
Higher number of Global priority is assumed to be Higher prior Important image.
if end date of due date is less than 24 hrs from the reference time offset then it is going to assured tasking.
Need offset from when scheduling is started. Example if scheduling is  needed from 10:00AM to 11 PM . Then offset is needed from 10:00 AM or 09:59 AM. Due date based assured tasking is the reason.
Iniial Camera Memory is needed at the start of the any oppr imaging/gsPass whichever is first. 
Iniial Readout Memory is needed at the start of the readout oppr.
Initial power is needed at start of the opportunity(imaging/gspass).For Now (since power constraint is not there for readout)
Iniial thermal value is needed before the start of the oppr imaging/gsPass/readout according to device (for NCCms:readout , for camera detector: Imaging ,For XBT : gs Pass oppr).
Eclipse Event should be starting from first oppr either gsPass/Imaging. Readout is happening after the first imaging so i guess not needed at readout.
'''
'''
Need offset from when scheduling is started. Example if scheduling is  needed from 10:00AM to 11 PM . Then offset is needed from 10:00 AM or 09:59 AM.
Iniial Memory,power and thermal value is needed at 10:00 AM
'''
def select_gs_pass_oppr(GS_pass_df,config):

    obj_preprocess = GSPassPreprocess(GS_pass_df)
    data = obj_preprocess.preprocess()

    obj_model = GSpassSelection(data,config)
    result,thermal_profile_gsPass = GSpasspostprocess(obj_model,data,config).get_gsPasses()# 21 seconds

    try :
        result['duration'] = result['end_time'] - result['start_time']
        result = result[result['duration']> 0]
    except:
        print("model is not converged or infeasible or not solved")

    return result
    

def select_img_opprtunity(image_opportunity_df,gs_pass_result_df,eclipse_df_dict,config):

    #basic flters
    #image_opportunity_df = image_opportunity_df[image_opportunity_df['OpportunityEndOffset']<config['scheduled_Hrs']*3600]
    image_opportunity_df = image_opportunity_df[image_opportunity_df['CloudCoverLimit']>image_opportunity_df['CloudCover']]
    image_opportunity_df = image_opportunity_df[image_opportunity_df['OffNadirLimit']>image_opportunity_df['OffNadir']]

    obj_preprocess = ImageAquisitionProcess(image_opportunity_df,gs_pass_result_df,eclipse_df_dict,config)
    data = obj_preprocess.preprocess()
    #print(data['cs1j2k2Domainlist__cs1j1k1'])

    #++++++++++++++++++++++++++  STEP 0  +++++++++++++++++++++++++++++++++++++++++++++++
    '''
    hard code some data
    '''
    data['camera_memory_capacity__s'] = {s:v for s,v in data['camera_memory_capacity__s'].items() }
    data['readout_memory_capacity__s'] = {s:v for s,v in data['readout_memory_capacity__s'].items() }
    data['power_capacity__s']  = {s:720000000 for s,v in data['power_capacity__s'].items() }
    data['initial_power_value__s']  = {s:v*0.3 for s,v in data['power_capacity__s'].items() }
    #++++++++++++++++++++++++++  STEP 1  +++++++++++++++++++++++++++++++++++++++++++++++
    config['objective']['GS_Pass_time'] = True
    config['objective']['total_priority'] = False
    config['objective']['total_readout_memory'] = False
    obj_model = ImageCapturePlan(data,config)

    #Readout Schedule 
    # data['GS_Pass_time_objective'] = obj_model.prob.objective.value()
    # config['objective']['GS_Pass_time'] = False
    # config['objective']['total_priority'] = False
    # config['objective']['total_readout_memory'] = True
    # obj_model = ImageCapturePlan(data,config)

    #++++++++++++++++++++++++++  STEP 2  +++++++++++++++++++++++++++++++++++++++++++++++

    # config['objective']['GS_Pass_time'] = False
    # config['objective']['total_priority'] = True
    # #config['objective']['total_readout_memory'] = False

    # obj_model = ImageCapturePlan(data,config)

    # data['total_priority_objective'] = obj_model.prob.objective.value()
    #++++++++++++++++++++++++++  STEP 3  +++++++++++++++++++++++++++++++++++++++++++++++

    # config['objective']['total_priority'] = False
    # config['objective']['total_readout_memory'] = True

    # obj_model = ImageCapturePlan(data,config)
    #++++++++++++++++++++++++++  PostProcess  +++++++++++++++++++++++++++++++++++++++++++++++
    post_obj = ImagecapturePostProcess(obj_model,data)
    img_capture_result= post_obj.get_schedule()
    #.isnull().sum()
    return img_capture_result,data
    

    #======================================================================================================================================================================================================

    pass

def get_aps_success_metric(img_capture_result,data):
    after_aps_plan_df = img_capture_result[img_capture_result['operation']=='Imaging']
    criteria_list = ['total_opprtunities_ratio','GP_ratio','LP_ratio','conflictImg_gsPass']

    total_conflict_images_list= [data['success_metric_before']['conflict_images']]
    GP_before = data['success_metric_before']['original_Total_GP']
    LP_before = data['success_metric_before']['original_Total_LP']
    TOppr_before = data['success_metric_before']['total_opportunities'] 
    before_list = [TOppr_before,GP_before,LP_before,total_conflict_images_list]

    GP_after= after_aps_plan_df['mean_global_priority'].sum()
    LP_after = after_aps_plan_df['mean_local_priority'].sum()
    TOppr_after= after_aps_plan_df['encoded_strip_id'].nunique()
    fraction_conflict_images_list = [list(after_aps_plan_df[after_aps_plan_df['encoded_strip_id'].isin(total_conflict_images_list)]['encoded_strip_id'].unique())]
    after_list = [TOppr_after,GP_after,LP_after,fraction_conflict_images_list]

    APS_success_metric_df = pd.DataFrame({'criteria':criteria_list,'potential_input':before_list,'APS_selected':after_list})
    #APS_success_metric_df['percentage'] = APS_success_metric_df['APS_selected'] / APS_success_metric_df['potential_input']
    APS_success_metric_df1 = APS_success_metric_df[:-1]
    APS_success_metric_df1['percentage'] = APS_success_metric_df1['APS_selected'] / APS_success_metric_df1['potential_input'] * 100
    APS_success_metric_df2 = APS_success_metric_df[-1:]

    APS_success_metric_df = pd.concat([APS_success_metric_df1,APS_success_metric_df2])

    return APS_success_metric_df
    

def get_downlink_schedule(image_downlink_df,img_capture_result,config):

    downlink_operation_list  = ['downlinking_from_camera','downlinking_from_Readout']
    img_capture_result_downlink = img_capture_result[img_capture_result['operation'].isin(downlink_operation_list)]
    DownlinkingPreProcessObj = DownlinkingPreProcess(image_downlink_df,img_capture_result_downlink,config)
    data_downlink = DownlinkingPreProcessObj.preprocess()
    
    if config['downlink_schedule_OnlyJustsortImgID']:
        downlink_result = pd.DataFrame(data_downlink['LP_DD_Priority_imgID'].items(),columns=['ImageID','computed_priority']).\
            sort_values(by='computed_priority',ascending=False)
    else:
        obj_downlink_model = ImageDownlinkPlan(data_downlink,config)
        downlink_result = ImageDownlinkPostProcess(obj_downlink_model,data_downlink).get_schedule()
        downlink_result = downlink_result[downlink_result['TileStripNo_downLoad']!=0]

    return downlink_result
    
def schedule():

    pass

def get_input_files(config):
    # GS PASS
    #GS_pass_df = pd.read_csv(config["csv_file_path"]["gs_pass_opportunity"])#APS_gsPasses_TV1#GS_Passes_mock1#GS_Passes_live1#GS_Passes_new (1)
    
    GS_pass_df = pd.read_csv("1_input_data/GS_Passes_new (1).csv")
    GS_pass_df['SatID'] = GS_pass_df['SatID'].astype(str)
    GS_pass_df['AOSOffset'] = GS_pass_df['AOSOffset'].astype(int)
    GS_pass_df['LOSOffset'] = GS_pass_df['LOSOffset'].astype(int)

    # image Opprtunity
    image_opportunity_df = pd.read_csv(config["csv_file_path"]["image_capture_opportunity"])#Imaging_mock1#APS_imagingOpportunities_TV1#Imaging_live#Imaging_new (1)
    # change made priority ulta
    #image_opportunity_df['Priority'] = 1/image_opportunity_df['Priority']
    
    image_opportunity_df['SatID'] = image_opportunity_df['SatID'].astype(str)
    image_opportunity_df['OpportunityStartOffset'] = image_opportunity_df['OpportunityStartOffset'].astype(int)
    image_opportunity_df['OpportunityEndOffset'] = image_opportunity_df['OpportunityEndOffset'].astype(int)
    image_opportunity_df_copy = image_opportunity_df.copy()
    image_opportunity_df_copy['X'] = image_opportunity_df_copy[['OpportunityStartTime','OpportunityStartOffset']].apply(lambda a: pd.to_datetime(a['OpportunityStartTime']) - pd.DateOffset(seconds=a['OpportunityStartOffset']),axis=1)
    image_opportunity_df_copy['Y'] = image_opportunity_df_copy[['OpportunityEndTime','OpportunityEndOffset']].apply(lambda a: pd.to_datetime(a['OpportunityEndTime']) - pd.DateOffset(seconds=a['OpportunityEndOffset']),axis=1)
    base_time_stamp = image_opportunity_df_copy["X"].to_list()[0]
    config['base_time_stamp_downlink'] = base_time_stamp

    #image Downlink
    image_downlink_df = pd.read_csv(config["csv_file_path"]["image_downlink_file"])
    image_downlink_df['assured_downlink_flag'] = [0,0] +[0]*(len(image_downlink_df)-2)
    image_downlink_df['delivery_type'] = 'standard_delivery' # expedited_delivery,super_expedited_delivery
    union_list_of_sat = list(set(image_opportunity_df['SatID']).union(set(GS_pass_df['SatID'])).union(set(image_downlink_df['SatID'])))
    hrs = config['scheduled_Hrs']

    # get dummy eclipse data close to reality
    #eclipse_df = pd.DataFrame()
    #for sat in satellite_list:
        #this_eclipse_df = eclipse_event_df[eclipse_event_df['SatID']==sat]
        #that_eclipse_df = get_eclipse_data(this_eclipse_df,config)
        #eclipse_df = pd.concat([that_eclipse_df,eclipse_df])
        
    min_time_index= min([image_opportunity_df['OpportunityStartOffset'].min(),image_opportunity_df['OpportunityEndOffset'].max(),GS_pass_df['AOSOffset'].min(),GS_pass_df['LOSOffset'].max()])
    max_time_index= max([image_opportunity_df['OpportunityStartOffset'].min(),image_opportunity_df['OpportunityEndOffset'].max(),GS_pass_df['AOSOffset'].min(),GS_pass_df['LOSOffset'].max()])

    hrs = (max_time_index - min_time_index)/3600
    hrs = math.ceil(hrs)
    while True:
        hrs += 1
        if hrs % 1.5==0:
            break


    in_orbit_eclipse_event = [1 for i in range(int(1.5*3600*0.6))] + [0 for i in range(int(1.5*3600*0.4))] #
    eclipse_df  = pd.DataFrame({'time_index': [i for i in range(min_time_index,min_time_index+hrs*3600)] ,"eclipse" : in_orbit_eclipse_event*int(hrs/1.5)})
    eclipse_df['SatID']= [union_list_of_sat] *len(eclipse_df)
    eclipse_df = eclipse_df.explode('SatID')
    eclipse_df_dict = {s: eclipse_df[eclipse_df['SatID']==s] for s in eclipse_df['SatID'].unique()}

    #eclipse_df = pd.DataFrame()
    #for sat in satellite_list:
        #this_eclipse_df = eclipse_event_df[eclipse_event_df['SatID']==sat]
        #that_eclipse_df = get_eclipse_data(this_eclipse_df,config)
        #eclipse_df = pd.concat([that_eclipse_df,eclipse_df])
   #eclipse_df_dict = {s: eclipse_df[eclipse_df['SatID']==s] for s in eclipse_df['SatID'].unique()}

    
    # get dummy data for assured tasking
    image_opportunity_df['encoded_stripId'] =   image_opportunity_df['StripID'].astype(str)+ '_' + image_opportunity_df['AoiID'].astype(str)
    total_capture_list = list(image_opportunity_df['encoded_stripId'].unique())
    no_of_list = len(total_capture_list)
    assured_capture_df = pd.DataFrame({'encoded_stripId':total_capture_list,'assured_task':[0,0]+[0]*(no_of_list-2)})
    image_opportunity_df = pd.merge(image_opportunity_df,assured_capture_df,on='encoded_stripId',how='left')
    image_opportunity_df = image_opportunity_df.drop(columns=['encoded_stripId'])

    # further processing eclipse data to align with gs pass where entire gs pass is assumed to be in eclipse region
    gsPassInput_df_copy = pd.read_csv(config["csv_file_path"]["gs_pass_opportunity"])
    gsPassInput_df_copy['SatID'] = gsPassInput_df_copy['SatID'].astype(str)
    gsPassInput_df_copy['AOSOffset'] = gsPassInput_df_copy['AOSOffset'].astype(int)
    gsPassInput_df_copy['LOSOffset'] = gsPassInput_df_copy['LOSOffset'].astype(int)
    gsPassInput_df_copy['list'] =  gsPassInput_df_copy[['AOSOffset','LOSOffset']].apply(lambda a : [i for i in range(a['AOSOffset'],a['LOSOffset']+1)],axis =1 )

    gsPassInput_df_copy1 = gsPassInput_df_copy[['SatID','list']]
    gsPassInput_df_copy1 = gsPassInput_df_copy1.explode('list')
    gsPassInput_df_grouped_copy1 = gsPassInput_df_copy1.groupby('SatID').agg(time_index_list = ('list',list)).reset_index()
    gsPasstimeIndexList__s = dict(zip(gsPassInput_df_grouped_copy1['SatID'],gsPassInput_df_grouped_copy1['time_index_list']))
    for k,v in eclipse_df_dict.items():
        if k in gsPasstimeIndexList__s.keys():
            this_time_index_list = gsPasstimeIndexList__s[k]
            v.loc[v["time_index"].isin(this_time_index_list), "eclipse"] = 1
            eclipse_df_dict[k] = v

    return {
            'GS_pass_df':GS_pass_df,\
            'image_opportunity_df':image_opportunity_df,\
            'image_downlink_df':image_downlink_df,\
            "eclipse_df_dict": eclipse_df_dict,
            "config":config
            }
    
if __name__ == "__main__":

    # Open and read the JSON file
    with open('1_input_data/config.json', 'r') as file:
        config = json.load(file)

    # if memory constraint False then thermal_constraint is also False
        config['constraints']['thermal_constraint_readout'] = config['constraints']['memory_constrant'] and config['constraints']['thermal_constraint_readout']
        config['constraints']['thermal_constraint_imaging'] = config['constraints']['memory_constrant'] and config['constraints']['thermal_constraint_imaging']

    #======================================================================================================================================================================================================
    # read_input
    input_dict = get_input_files(config)
    config = input_dict['config']
    #======================================================================================================================================================================================================
    #gs pass_selection
    gs_pass_result_df = select_gs_pass_oppr(input_dict['GS_pass_df'],config)
    gs_pass_result_df['Eclipse'] = 1 ## dummy
    gs_pass_result_df['duration'] = gs_pass_result_df['end_time'] - gs_pass_result_df['start_time']
    gs_pass_result_df = gs_pass_result_df[gs_pass_result_df['duration']> 0]
    interpret_gs_pass_result_df_copy = gs_pass_result_df.copy()# this not the gsPass result as it is to be get filtered after due to other factors in image capture plan.It is just to get require info in interpret result.

    print("image_capture_plan_starting")
    #======================================================================================================================================================================================================
    #image_selection
    img_capture_result,capture_plan_data_input= select_img_opprtunity(input_dict['image_opportunity_df'],gs_pass_result_df,input_dict['eclipse_df_dict'],config)
    img_capture_result['base_time'] = config['base_time_stamp_downlink']
    #img_capture_result = img_capture_result[img_capture_result['operation']=='Imaging']
    #readout_result = img_capture_result[img_capture_result['operation']=='Readout']
    interpret_img_capture_resul_copy = img_capture_result.copy()
    #======================================================================================================================================================================================================
    # get APS success metrics 
    APS_success_metric_df = get_aps_success_metric(img_capture_result,capture_plan_data_input)
    #======================================================================================================================================================================================================
    print("Downlink_plan_starting")
    try:
        print(dfd)
        downlink_result = get_downlink_schedule(input_dict['image_downlink_df'],img_capture_result,config)
    except:
        print("downlink_schedule_has_some_error")
        downlink_result = pd.DataFrame()
    downlink_result['base_time'] = config['base_time_stamp_downlink']
    #======================================================================================================================================================================================================
    #img_capture_result[img_capture_result['download_from_']]
    gs_pass_result_df.to_csv("5_output_data/gs_pass_result_df.csv",index=None)
    img_capture_result.to_csv("5_output_data/img_capture_schedule.csv",index=None)
    APS_success_metric_df.to_csv("5_output_data/APS_success_metric.csv",index = None)
    downlink_result.to_csv("5_output_data/downlink_result.csv",index = None)

    interpret_image_opportunity_df = pd.read_csv(config["csv_file_path"]["image_capture_opportunity"])
    interpret_result_dict = interpret_result(interpret_image_opportunity_df,interpret_gs_pass_result_df_copy,interpret_img_capture_resul_copy,config)
    for k,v in interpret_result_dict.items():
        v['base_time'] = config['base_time_stamp_downlink']
        v.to_csv("5_output_data/"+k+".csv",index = None)

    
    only_img_capture_result = img_capture_result[img_capture_result['operation']=='Imaging'][['SatID','start_time','end_time','AoiID','StripID','base_time']]
    only_img_capture_result['start_time'] = only_img_capture_result[['start_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['start_time']),axis=1)
    only_img_capture_result['end_time'] = only_img_capture_result[['end_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['end_time']),axis=1)

    only_readout_result = img_capture_result[img_capture_result['operation']=='Readout'][['SatID','start_time','end_time','base_time']]
    only_readout_result['start_time'] = only_readout_result[['start_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['start_time']),axis=1)
    only_readout_result['end_time'] = only_readout_result[['end_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['end_time']),axis=1)

    result_dict = {"only_readout_result":only_readout_result,\
                  "only_img_capture_result":only_img_capture_result}
    result_dict.update(interpret_result_dict)

    #print(only_img_capture_result,only_readout_result,downlink_result)
#config['constraints'] = ['Thermal_constraints']